## Compare CBC and Gurobi

Compare the computation time of the CBC and Gurobi solvers for the same scenarios

In [3]:
import logging
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
import numpy as np
import random
import seaborn as sns
import pandas as pd
import statistics as stat
import os
import yaml
import glob

ModuleNotFoundError: No module named 'matplotlib'

### 1 Load Data

In [2]:
data = {"AGVs": [], 
        "randseed": [], 
        "solver": [],
        "delay": [],         
        "horizon": [], 
        "total_time": [], 
        "comp_time_vec": [],
        "comp_time_avg": [],
        "comp_time_max": []}

yaml_list = glob.glob("ICAPS_solver_comp/*.yaml")
horizon_0_data = {"AGVs": [], "randseed": [], "solver": [], "delay": [], "total_time": []}

for file in yaml_list:
    split_filename = file.split("_")
    horizon = str(split_filename[-1].split(".")[0])
    delay = str(split_filename[-3])
    seed = str(split_filename[-5])
    AGVs = str(split_filename[-7])
    with open(file, "r") as stream:
        try:
            yaml_data = yaml.safe_load(stream)
            cumulative_time = yaml_data["results"]["total time"]
            comp_time_vec = yaml_data["results"]["comp time"]["solve_time"]
            comp_time_avg = yaml_data["results"]["comp time"]["avg"]
            comp_time_max = yaml_data["results"]["comp time"]["max"]
            solver = yaml_data["parameters"]["solver"]
            data["AGVs"].append(int(AGVs))
            data["randseed"].append(int(seed))
            data["solver"].append(solver)
            data["delay"].append(int(delay))
            data["horizon"].append(int(horizon))
            data["total_time"].append(int(cumulative_time))
            data["comp_time_vec"].append(comp_time_vec)
            data["comp_time_avg"].append(comp_time_avg)
            data["comp_time_max"].append(comp_time_max)

        except yaml.YAMLError as exc:
            print(exc)
            
columns = ["AGVs", 
           "randseed", 
           "solver",
           "delay", 
           "horizon", 
           "total_time", 
           "comp_time_vec", 
           "comp_time_avg", 
           "comp_time_max"]

df = pd.DataFrame(data, columns=columns)
print(df)

     AGVs  randseed solver  delay  horizon  total_time  \
0      60         7    GRB     10       10        5046   
1      50        12    CBC     10        1        3534   
2      50        14    CBC     10        1        4007   
3      60        16    GRB     10        1        4874   
4      60         7    CBC     10        1        5166   
..    ...       ...    ...    ...      ...         ...   
149    50         6    CBC     10        2        3460   
150    50         9    CBC     10       15        3829   
151    50        19    CBC     10        1        3534   
152    50         1    GRB     10        2        3644   
153    50        13    GRB     10       10        3435   

                                         comp_time_vec  comp_time_avg  \
0    [[1.1526555939999952, 1.1490951370000033, 0.81...       0.333874   
1    [[0.013490760000003377, 0.028106063999999265, ...       0.044818   
2    [[0.0994577019999987, 0.21463387400000045, 0.0...       0.065019   
3    [[0.01

### 2 Compare Gurobi and CBC times

In [3]:
df_CBC = df[df.solver == "CBC"]
df_GRB = df[df.solver == "GRB"]

# print(df_CBC)
# print(df_GRB)

# dataframe for comparison
compdata = {"AGVs": [], 
            "randseed": [], 
            "delay": [], 
            "horizon": [], 
            "time_CBC": [], 
            "time_GRB": [], 
            "factor": []}

no_match_count = 0

# loop through all scenarios solved with CBC
for index, row in df_GRB.iterrows():
    AGVs = row["AGVs"]
    randseed = row["randseed"]
    delay = row["delay"]
    horizon = row["horizon"]
    total_time_GRB = row["comp_time_max"]
    try: 
        # find corresponding scenario in CBC dataset
        CBC_data = df_CBC[(df_CBC.AGVs == AGVs) & (df_CBC.randseed == randseed) & (df_CBC.horizon == horizon) & (df_CBC.delay == delay)].iloc[0]
        
        total_time_CBC = float(CBC_data["comp_time_max"])
        factor = float(total_time_CBC)/float(total_time_GRB)
        
        compdata["AGVs"].append(int(AGVs))
        compdata["randseed"].append(int(randseed))
        compdata["delay"].append(int(delay))
        compdata["horizon"].append(int(horizon))
        compdata["time_CBC"].append(float(total_time_CBC))
        compdata["time_GRB"].append(float(total_time_GRB))
        compdata["factor"].append(float(factor))
    except IndexError:
        # no match found
        no_match_count += 1
        continue
        
print("no match count: {}".format(no_match_count))
# print(compdata["factor"])
for row in compdata:
    
    print(row)
    print(compdata[row])
#     AGVs = compdata[row]
#     print(AGVs)
#     randseed = row[1]
#     delay = row[2]
#     horizon = row[3]
#     factor = row[4]
#     statss = AGVs + " " + randseed + " " + delay + " " + horizon 
#     print(row)
#     print(AGVs)
#     print(" - {number:.0f} - {stats}".format(number=factor, stats=statss))
    

no match count: 0
AGVs
[60, 60, 50, 60, 60, 50, 60, 50, 60, 60, 50, 50, 60, 50, 50, 50, 60, 50, 60, 60, 50, 60, 50, 60, 60, 50, 60, 50, 50, 50, 60, 50, 50, 60, 60, 50, 60, 50, 60, 60, 50, 50, 60, 50, 50, 60, 50, 50, 50, 60, 60, 60, 50, 50, 50, 60, 60, 60, 60, 50, 60, 50, 60, 50, 50, 60, 50, 50, 50, 50, 60, 50, 50, 50, 50]
randseed
[7, 16, 3, 3, 18, 14, 13, 14, 13, 6, 6, 11, 20, 13, 3, 5, 1, 10, 2, 9, 4, 1, 9, 17, 19, 7, 13, 12, 1, 5, 11, 20, 2, 5, 6, 9, 10, 6, 3, 3, 16, 3, 5, 11, 7, 2, 19, 5, 18, 1, 7, 4, 5, 17, 6, 14, 12, 4, 5, 7, 6, 1, 15, 11, 5, 8, 4, 15, 8, 9, 5, 13, 2, 1, 13]
delay
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
horizon
[10, 1, 5, 1, 1, 10, 1, 1, 10, 5, 2, 10, 1, 15, 2, 10, 2, 1, 2, 1, 5, 1, 1, 1, 1, 10, 15, 1, 5, 2